In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sea
from tqdm.notebook import tqdm
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import cv2
import wandb

from sklearn.datasets import fetch_openml
from scipy.io import loadmat

device = 'cuda' if torch.cuda.is_available() else 'cpu'

plt.style.use('seaborn')
np.__version__, device

('1.21.2', 'cuda')

In [11]:
PATH = '/scratch/fk/double_mnist'

def load_data(PATH, split):
    files, outcome = [], []
    
    for folder in tqdm(os.listdir(os.path.join(PATH, split))):
        if folder[0] == folder[1]:
            continue
        
        for file in os.listdir(os.path.join(PATH, split, folder)):
            files.append(os.path.join(PATH, split, folder, file))
            outcome.append(folder)
            
    return pd.DataFrame({
        'filename': files,
        'outcome': outcome
    }).sample(frac=1.0)
    
df_train = load_data(PATH, 'train')
df_val = load_data(PATH, 'val')
df_test = load_data(PATH, 'test')

df_train.shape, df_val.shape, df_test.shape

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

((58000, 2), (14000, 2), (18000, 2))

In [12]:
df = pd.concat([df_train, df_val, df_test], axis=0).sample(frac=1.0)
df.shape

(90000, 2)

In [13]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df, test_size=0.2, stratify=df['outcome'])
df_val, df_test = train_test_split(df_val, test_size=0.5, stratify=df_val['outcome'])

df_train.shape, df_val.shape, df_test.shape

((72000, 2), (9000, 2), (9000, 2))

In [14]:
def load_image(file):
    img = plt.imread(file)
    img = cv2.resize(img, (32, 32))
    return img


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0, 1)
])

class MultiDigitDataset:
    def __init__(self, df, transform=None):
        self.df = df
        self.n_samples = len(df)
        self.transform = transform 
        
    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        x = load_image(self.df.iloc[idx, 0])
        if self.transform:
            x = self.transform(x)
            
        y = self.df.iloc[idx, 1]
        y_one_hot = np.zeros(10, dtype=np.float32)
        y_one_hot[[int(y[0]), int(y[1])]] = 1
        
        return x, torch.tensor(y_one_hot)
        

In [15]:
train_dataset = MultiDigitDataset(df_train, transform)
val_dataset = MultiDigitDataset(df_val, transform)
test_dataset = MultiDigitDataset(df_test, transform)

batch_size = 256

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [16]:
a, b = next(iter(train_loader))

a.shape, b.shape

(torch.Size([256, 1, 32, 32]), torch.Size([256, 10]))

In [23]:
class MlpModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.ModuleList(
            [
                self.get_block(1024, 512),
                self.get_block(512, 256),
                self.get_block(256, 128),
                self.get_block(128, 128),
                self.get_block(128, 64)
                
            ]
        )
        
        self.drop = nn.Dropout(0.2)
        self.classiifer = nn.Linear(64, 10)
    
    def get_block(self, in_c, out_c):
        return nn.Sequential(
            nn.Linear(in_c, out_c, bias=False),
            nn.BatchNorm1d(out_c),
            nn.ReLU(),
            nn.Dropout(0.2)
        )    
        
    def forward(self, x):
        x = x.view(-1, 32*32)
        
        for i, layer in enumerate(self.layers):
            x = layer(x)
            
        return self.classiifer(x)
    


In [24]:
from sklearn.metrics import accuracy_score

class Trainer:
    def __init__(self,
                 model,
                 train_loader,
                 val_loader,
                 device,
                 loss_fxn, 
                 logger,
                 params):

        self.device = device
        self.params = params        
        self.model = model.to(self.device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.loss_fxn = loss_fxn
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.params['lr'])
        self.logger = logger
        
    def training_step(self, x, y):
        y_pred = self.model(x)
        loss = self.loss_fxn(y_pred, y)
        y_pred_bin = (y_pred > 0.5).to(torch.int64)
        acc = accuracy_score(y_pred_bin.detach().cpu(), y.detach().cpu())
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss, acc
    
    def val_step(self, x, y):
        with torch.no_grad():
            y_pred = self.model(x)
            
        loss = self.loss_fxn(y_pred, y)
        y_pred_bin = (y_pred > 0.5).to(torch.int64)
        acc = accuracy_score(y_pred_bin.detach().cpu(), y.detach().cpu())
        
        return loss, acc
    
    def go_over_one_batch(self, loader, step_fxn):
        loss, acc = 0, 0
        for x, y in tqdm(loader):
            x, y = x.to(self.device), y.to(self.device)
            l, a = step_fxn(x, y)
            loss, acc = loss + l, acc + a
            
        return loss/len(loader), acc/len(loader)
    
    def train(self, epochs = 10):
        for epoch in tqdm(range(epochs)):
            
            train_loss, train_acc = self.go_over_one_batch(self.train_loader, self.training_step)
            val_loss, val_acc = self.go_over_one_batch(self.val_loader, self.val_step)
        
            print(f"[Epoch: {epoch}] Training:[loss:{train_loss:.4f} acc:{train_acc:.3f}] Val:[loss:{val_loss:.4f} acc:{val_acc:.3f}]" )
            if self.logger:
                self.logger.log({
                    'train_loss':train_loss,
                    'val_loss':val_loss,
                    'train_acc': train_acc,
                    'val_acc': val_acc
                })
            
        if self.logger:
            self.logger.finish()
            

In [25]:
params = {
    'lr' : 1e-3,
    'batch_size':512,
    'epoch': 1000,
}


wandb.init(
    project="SMAI-A3-double-mnist-ANN",
    config=params,
    name=f"4-layer|tanh"
    
)

trainer = Trainer(
    model = MlpModel(),
    train_loader = train_loader,
    val_loader = val_loader, 
    device = device, 
    loss_fxn = nn.BCEWithLogitsLoss(),
    logger=None,
    params = params
)

wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [26]:
trainer.train()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/282 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

[Epoch: 0] Training:[loss:0.4650 acc:0.001] Val:[loss:0.4043 acc:0.004]


  0%|          | 0/282 [00:00<?, ?it/s]

...

  0%|          | 0/282 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

[Epoch: 4] Training:[loss:0.2577 acc:0.215] Val:[loss:0.2649 acc:0.228]


  0%|          | 0/282 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def test_model(model, loader):
    acc = 0
    for x, y in tqdm(loader):
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            y_pred = model(x)
            y_pred_bin = (y_pred > 0.5).to(torch.int64)
            acc += accuracy_score(y_pred_bin.cpu(), y.cpu())
        
    return acc/len(loader)

test_model(trainer.model, test_loader)

  0%|          | 0/36 [00:00<?, ?it/s]

0.4232204861111111

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.params = params
                
        self.layers = nn.ModuleList(
            [
                self.get_block(1, 32, kernel_size=self.params['kernel_size'], padding=self.params['padding']), # 32, 16, 16
                self.get_block(32, 64, kernel_size=self.params['kernel_size'], padding=self.params['padding']), # 64, 8, 8 
                self.get_block(64, 128, kernel_size=self.params['kernel_size'], padding=self.params['padding']), # 128, 4, 4
                self.get_block(128, 256, kernel_size=self.params['kernel_size'], padding=self.params['padding']), # 256, 2, 2
            ]
        )
        
        self.fc1 = nn.Linear(256 * 2 * 2, 256 * 1 * 1)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.drop = nn.Dropout(0.2)
        
    def get_block(self, in_c, out_c, kernel_size, padding):
        return nn.Sequential(
            nn.Conv2d(in_c, in_c, kernel_size=kernel_size, stride=1, padding=padding, bias=not self.params['batchnorm']),
            nn.BatchNorm2d(in_c) if self.params['batchnorm'] else nn.Identity(),
            nn.ReLU(),
            
            nn.Conv2d(in_c, out_c, 3, 1, 1, bias=True),
            nn.ReLU(),
            
            nn.Conv2d(out_c, out_c, 3, 1, 1, bias=not self.params['batchnorm']),
            nn.BatchNorm2d(out_c) if self.params['batchnorm'] else nn.Identity(),
            nn.ReLU(),
            
            nn.MaxPool2d(2),
            nn.Dropout(0.2) if self.params['dropout'] else nn.Identity()
        )
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            
        x = x.view(-1, 256*2*2)
        x = self.drop(nn.LeakyReLU()(self.fc1(x)))
        x = self.drop(nn.LeakyReLU()(self.fc2(x)))
        return self.fc3(x)
        
        

In [9]:


params = {
    'lr' : 7e-4,
    'batch_size':256,
    'epoch': 10,
    'dropout': True,
    'batchnorm': True,
    'kernel_size': 3,
    'padding': 1
}


wandb.init(
    project="SMAI-A3-double-mnist-CNN",
    config=params,
    name=f"batch_size={256}"
    
)

train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)


trainer = Trainer(
    model = CNNModel(params),
    train_loader = train_loader,
    val_loader = val_loader, 
    device = device, 
    loss_fxn = nn.BCEWithLogitsLoss(),
    logger=None,
    params=params
)

trainer.train()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

[Epoch: 0] Training:[loss:0.4469 acc:0.004] Val:[loss:0.3617 acc:0.020]


...

[Epoch: 8] Training:[loss:0.0157 acc:0.970] Val:[loss:0.0188 acc:0.966]


  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

[Epoch: 9] Training:[loss:0.0145 acc:0.972] Val:[loss:0.0152 acc:0.972]


train_acc,▁▂▆███████
train_loss,█▅▂▁▁▁▁▁▁▁
val_acc,▁▄▇███████
val_loss,█▄▂▁▁▁▁▁▁▁
train_acc,0.97201
train_loss,0.0145
val_acc,0.97174
val_loss,0.01522
